In [6]:
#SET UP
#import sys
#sys.path
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel
from survival import MIRModel

In [3]:
#PROCESS INPUTS
df = pd.read_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality.csv')

#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]

#set model parameters
df['disease_period'] = 10
df['slope'] = .21

df.head()

,Unnamed: 0,location_id,year_id,sex_id,age_group_id,acause,mi_ratio,life_table_parameter_id,background_mortality,disease_period,slope
0,1,6,1980,1,1,neo_bladder,0.296086,3.0,0.076672,10,0.21
1,2,6,1980,1,1,neo_brain,0.670154,3.0,0.076672,10,0.21
2,3,6,1980,1,1,neo_breast,0.486486,3.0,0.076672,10,0.21
3,4,6,1980,1,1,neo_colorectal,0.560212,3.0,0.076672,10,0.21
4,5,6,1980,1,1,neo_esophageal,0.701150,3.0,0.076672,10,0.21


In [ ]:
#RUN BASE MODEL
num_years = 5

model = MIRModel(df['mi_ratio'],
                 df['other_mortality'],
                 disease_period=10)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df['excess_mortality_og_mir_model'] = model.excess_mortality
df['abs_survival_rate_og_mir_model'] = survival_rate['abs']
df['rel_survival_rate_og_mir_model'] = survival_rate['rel']

In [ ]:
#RUN DECAY MODEL
num_years = 5

model = DecayMIRModel(df['mi_ratio'],
                 df['other_mortality'],
                 df['disease_period'],
                 df['slope'])
model.compute_base_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

df['base_excess_mortality_decay_model'] = model.base_excess_mortality
df['abs_survival_rate_decay_model'] = survival_rate['abs']
df['rel_survival_rate_decay_model'] = survival_rate['rel']

In [4]:
#RUN THREE PARAMETER LOG MODEL
parameters = pd.read_csv('/homes/fed1/Survival_Franny/three_parameter_log_form_values_SEER_5_4_2021.csv')

# all cause parameters for testing
df['a'] = parameters[parameters['cause_name']=="All Sites"]

# later/ to do: merge in by cause (need acauses)

In [ ]:
#READ IN AND APPEND OLD SURVIVAL PREDICTIONS TO COMPARE

In [ ]:
df.head()

In [ ]:
df.to_csv('/home/j/temp/fed1/comparing_new_decay_method_4_28_2021.csv')